In [7]:
import sys
import os

parent = os.path.dirname(os.path.realpath("../../"))
sys.path.append(parent)

import numpy as np
import scipy
import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import glob

from core import *
from utils import phantom_builder
from utils import geometry

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Make Phantom

In [8]:
voxel_size = np.array([0.0005, 0.0005, 0.0005])
surface_mesh = o3d.io.read_triangle_mesh(
    f"{parent}/assets/cardiac_TEE_phantom/esophageal_surface.ply"
)
body_mask = phantom_builder.voxelize(voxel_size[0], mesh=surface_mesh)

In [ ]:
body_mask.shape

In [ ]:
test_phantom = phantom.Phantom(
    voxel_dims=(voxel_size[0], voxel_size[0], voxel_size[0]),
    matrix_dims=body_mask.shape,
    baseline=(1540, 1000),
    seed=None,
)

blood = tissue.Tissue(name="blood", c=1578, rho=1060, sigma=1.3, scale=0.00001, label=1)
myocardium = tissue.Tissue(
    name="myocardium", c=1592, rho=1081, sigma=20, scale=0.0001, label=2
)
esophagus = tissue.Tissue(
    name="esophagus", c=1500, rho=1100, sigma=10, scale=0.0001, label=3
)
fat = tissue.Tissue(name="fat", c=1480, rho=970, sigma=15, scale=0.0001, label=4)

kidney_file_dir = f"{parent}/assets/cardiac_TEE_phantom/"
kidney_tissue_list = [blood, myocardium, esophagus]

test_phantom.build_organ_from_mesh(
    surface_mesh, voxel_size[0], kidney_tissue_list, dir_path=kidney_file_dir
)
test_phantom.set_default_tissue(fat)

In [ ]:
test = test_phantom.get_complete()

In [ ]:
index = 200
plt.imshow(test[0, :, :, index], cmap="gray", vmin=1450, vmax=1650)
plt.imshow(
    body_mask[:, :, index], alpha=(body_mask[:, :, index] > 0) * 0.9, cmap="Reds"
)

In [ ]:
plt.imshow(
    test_phantom.get_complete()[0, 200, :, ::-1].T, vmin=1450, vmax=1650, cmap="gray"
)

In [ ]:
for i in range(0, test_phantom.get_complete().shape[1], 4):
    plt.imshow(
        test_phantom.get_complete()[0, i, :, ::-1].T, vmin=1450, vmax=1650, cmap="gray"
    )
    plt.savefig(f"video/heart_phantom_{str(i).zfill(3)}.png", dpi=150)

# Set up simulation

In [ ]:
num_transducers = 5

transducers = [
    transducer.Focused(
        max_frequency=2e6,
        elements=128,
        width=15e-3,
        height=15e-3,
        sensor_sampling_scheme="not_centroid",
        sweep=(np.pi / 2, np.pi / 2),
        ray_num=(128, 128),
        imaging_ndims=3,
        focus_azimuth=70e-3,
        focus_elevation=100e-3,
        balance_3D=True,
    )
    for i in range(num_transducers)
]


for t in transducers:
    t.make_sensor_coords(1540)  # test_phantom.baseline[0]

test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

In [ ]:
# Mid esophageal view
pt, normal = test_transducer_set.place_on_mesh_voxel(
    0, surface_mesh, [150, 270, 200], voxel_size[0]
)
transducer_pose = geometry.Transform(
    rotation=(np.pi * 0, np.pi * 0, np.pi * 0), translation=pt
)
test_transducer_set.assign_pose(0, transducer_pose)

# Transgastric
pt, normal = test_transducer_set.place_on_mesh_voxel(
    0, surface_mesh, [250, 250, 50], voxel_size[0]
)
transducer_pose = geometry.Transform(
    rotation=(np.pi * 0.25, np.pi * -0.4, np.pi * 0), translation=pt
)
test_transducer_set.assign_pose(1, transducer_pose)

# Deep transgastric
pt, normal = test_transducer_set.place_on_mesh_voxel(
    0, surface_mesh, [350, 400, 50], voxel_size[0]
)
transducer_pose = geometry.Transform(
    rotation=(np.pi * 0.25, np.pi * -0.7, np.pi * 0), translation=pt
)
test_transducer_set.assign_pose(2, transducer_pose)

# Mid-upper esophageal
pt, normal = test_transducer_set.place_on_mesh_voxel(
    0, surface_mesh, [100, 250, 320], voxel_size[0]
)
transducer_pose = geometry.Transform(
    rotation=(np.pi * 0.1, np.pi * 0, np.pi * 0), translation=pt
)
test_transducer_set.assign_pose(3, transducer_pose)

# High-upper esophageal
pt, normal = test_transducer_set.place_on_mesh_voxel(
    0, surface_mesh, [100, 250, 450], voxel_size[0]
)
transducer_pose = geometry.Transform(
    rotation=(np.pi * 0.25, np.pi * 0, np.pi * 0), translation=pt
)
test_transducer_set.assign_pose(4, transducer_pose)

In [ ]:
test_sensor = sensor.Sensor(
    transducer_set=test_transducer_set, aperture_type="transmit_as_receive"
)

In [ ]:
# test_transducer_set.plot_transducer_coords(scale = 0.1, phantom_coords = np.array((o3d.io.read_triangle_mesh(f"{parent}/assets/cardiac_TEE_phantom/03_esophagus.obj")).sample_points_uniformly(1000).points))
test_transducer_set.plot_transducer_coords()
test_transducer_set.plot_transducer_fovs()

In [ ]:
simprops = simulation.SimProperties(
    grid_size=(110e-3, 20e-3, 20e-3),
    voxel_size=(0.5e-3, 0.5e-3, 0.5e-3),
    PML_size=(16, 8, 8),
    PML_alpha=2,
    t_end=12e-5,  # [s]
    bona=6,  # parameter b/a determining degree of nonlinear acoustic effects
    alpha_coeff=0.5,  # [dB/(MHz^y cm)]
    alpha_power=1.5,
)

In [ ]:
test_experiment = experiment.Experiment(
    simulation_path=f"cardiac_TEE_2MHz",
    sim_properties=simprops,
    phantom=test_phantom,
    transducer_set=test_transducer_set,
    sensor=test_sensor,
    nodes=1,
    results=None,
    indices=None,
    workers=4,
    additional_keys=[],
    repeat=-1,
)

test_experiment.save()

In [ ]:
sensor_centroids = []
for i in range(len(test_transducer_set)):
    sensor_centroids.append(
        np.mean(
            test_sensor.sensor_coords[
                i * test_sensor.sensor_coords.shape[0] // len(test_transducer_set) : (
                    i + 1
                )
                * test_sensor.sensor_coords.shape[0]
                // len(test_transducer_set),
                :,
            ],
            axis=0,
        )
        / voxel_size
        + np.array(test.shape[1:]) / 2
    )

In [ ]:
test_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[0][2]))]
)

In [ ]:
test_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[1][2]))]
)

In [ ]:
test_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[2][2]))]
)

In [ ]:
test_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[3][2]))]
)

In [ ]:
test_experiment.visualize_sensor_mask(
    index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[4][2]))]
)

In [ ]:
test_experiment.plot_ray_path(32 // 2)

In [ ]:
test_experiment.plot_ray_path(32 * 1 + 32 // 2)

In [ ]:
test_experiment.plot_ray_path(32 * 2 + 32 // 2)

In [ ]:
test_experiment.plot_ray_path(32 * 3 + 32 // 2)

In [ ]:
test_experiment.plot_ray_path(32 * 4 + 32 // 2)

# Run Simulation

In [ ]:
test_experiment = experiment.Experiment.load(f"cardiac_TEE_2MHz")
test_experiment.run(dry=True)

In [ ]:
test_experiment.run(repeat=False)

# Reconstruction

In [4]:
test_experiment.add_results()

In [5]:
test_reconstruction = reconstruction.DAS(experiment=test_experiment)

In [ ]:
# for i,result in tqdm.tqdm(enumerate(test_reconstruction.results)):
#     if result[0].shape[0] != 2477:
#         print(result[0].shape)
#         prit(i)
#         break

In [ ]:
signals = test_reconstruction.get_signals(
    dimensions=3, matsize=128, downsample=1, workers=16, tgc=10
)

In [4]:
from utils import utils

In [14]:
for i, signal in enumerate(signals):
    # signal = np.where(np.isnan(signal), 0, signal)
    utils.save_mrc(signal * 0.01, f"output/cardiac_TEE_signals_{i}.mrc")

In [ ]:
image = test_reconstruction.get_image(
    dimensions=3, matsize=128, downsample=1, workers=16, tgc=10
)
# image = np.where(np.isnan(image), 0, image)
utils.save_mrc(image * 0.01, "output/cardiac_TEE_image.mrc")

In [ ]:
signals[0]

In [ ]:
assert False

In [ ]:
signals = test_reconstruction.get_signals(
    dimensions=2, matsize=128, downsample=0.5, workers=16, tgc=10
)

In [ ]:
plt.imshow(signals[0], vmax=20000, cmap="gray")

In [ ]:
plt.imshow(signals[1], vmax=20000, cmap="gray")

In [ ]:
plt.imshow(signals[2], vmax=20000, cmap="gray")

In [ ]:
plt.imshow(signals[3], vmax=20000, cmap="gray")

In [ ]:
plt.imshow(signals[4], vmax=20000, cmap="gray")